In [ ]:
import param
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv
import dask.dataframe as dd
import parambokeh
import cartopy.crs as ccrs
import xarray as xr
import cv2 as cv
import datashader as ds

from matplotlib.path import Path 
from colorcet import cm_n
from holoviews.streams import RangeXY, BoxEdit, PointDraw, PolyDraw, PolyEdit
from holoviews.operation.datashader import datashade, regrid

hv.extension('bokeh')

These demos require:
    
    * bokeh 0.12.14dev6
    * holoviews 1.10 (unreleased)
    * datashader 0.6.5
    * xarray 0.10.0
    * geoviews 1.5 (unreleased)
    * parambokeh
    * colorcet

## An example dashboard

In [ ]:
usecols = ['dropoff_x', 'dropoff_y', 'pickup_x', 'pickup_y', 'dropoff_hour']
df = dd.read_parquet('/Users/philippjfr/anacondaviz/data/nyc_taxi_wide.parq')
df = df[usecols].persist()


In [ ]:
usecols = ['dropoff_x', 'dropoff_y', 'pickup_x', 'pickup_y', 'dropoff_hour']
df = dd.read_parquet('/Users/philippjfr/anacondaviz/data/nyc_taxi_wide.parq')
df = df[usecols].persist()

url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
tiles = gv.WMTS(url, crs=ccrs.GOOGLE_MERCATOR)
tile_options = dict(width=600,height=400,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)

class NYCTaxiExplorer(hv.streams.Stream):
    alpha      = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap   = param.ObjectSelector(default=cm_n["fire"], objects=cm_n.values())
    hour       = param.Range(default=(0, 24), bounds=(0, 24))
    location   = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])

    def make_view(self, x_range, y_range, **kwargs):
        map_tiles = tiles.opts(style=dict(alpha=self.alpha), plot=tile_options)
        points = hv.Points(df, kdims=[self.location+'_x', self.location+'_y'], vdims=['dropoff_hour'])
        if self.hour != (0, 24): points = points.select(dropoff_hour=self.hour)
        taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=self.colormap,
                               dynamic=False, x_range=x_range, y_range=y_range, width=600, height=400)
        return map_tiles * taxi_trips

explorer = NYCTaxiExplorer(name="NYC Taxi Trips")
dmap = hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])
plot = hv.renderer('bokeh').get_plot(dmap)
parambokeh.Widgets(explorer, view_position='right', callback=explorer.event, plots=[plot])

In [ ]:
np.linspace(x_range)

In [ ]:
for 

## ROI Editor

In [ ]:
%%opts Curve [width=400] {+framewise} NdOverlay [legend_limit=0] 
%%opts Polygons (fill_alpha=0.2 line_color='white') VLine (color='black')

data = np.load('/Users/philippjfr/topographica/external/holoviews/examples/assets/twophoton.npz')
calcium_array = data['Calcium']
calcium_ds = hv.Dataset((np.arange(50), np.arange(111), np.arange(62), calcium_array),
                ['Time', 'x', 'y'], 'Fluorescence')

polys = hv.Polygons([])
box_stream = BoxEdit(source=polys)

def roi_curves(data):
    if not data:
        return hv.NdOverlay({0: hv.Curve([], 'Time', 'Fluorescence')})
    
    curves = {}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = calcium_ds.select(x=(x0, x1), y=(y0, y1))
        curves[i] = hv.Curve(selection.aggregate('Time', np.mean))
    return hv.NdOverlay(curves)

hlines = hv.HoloMap({i: hv.VLine(i) for i in range(50)}, 'Time')
dmap = hv.DynamicMap(roi_curves, streams=[box_stream])

calcium_ds.to(hv.Image, ['x', 'y'], dynamic=True) * polys + dmap * hlines

## Point Drawing

In [ ]:
%%opts Points [width=500 height=500] (size=10 line_color='black' nonselection_alpha=0.3)
points = hv.Points([(0, 0), (0.5, 0.5), (1, 0)])
point_stream = PointDraw(data=points.columns(), source=points)
trimesh = hv.DynamicMap(hv.TriMesh.from_vertices, streams=[point_stream])
trimesh * points

## GrabCut

In [ ]:
img = cv.imread('/Users/philippjfr/Downloads/hopper.jpg')[::6, ::6]
bounds = (0, 0, *img.shape[:2][::-1])
rgb = hv.RGB(img[:, :, ::-1], bounds=bounds)

def rasterize(data, img):
    """
    Given polygon data generate a mask of
    the same shape as the supplied img.
    """
    ny, nx = img.shape[:2]
    mask = False
    for xs, ys in zip(data.get('x', []), data.get('y', [])):
        if len(xs) < 3:
            continue
        path = Path(np.column_stack([xs, ys]))
        grid = path.contains_points(rgb.array([0, 1]))
        mask |= grid.reshape((nx,ny)).T
    return mask.astype(np.uint8)

def extract_foreground(fg_data, bg_data):
    """
    Given foreground and background polygon data,
    run the GrabCut algorithm and return a masked
    RGB of the foreground of the original Image.
    """
    if not fg_data:
        return hv.RGB(img[:, :, ::-1], bounds=bounds)
    fg_mask = rasterize(fg_data, img)
    bg_mask = rasterize(bg_data, img) if bg_data else False
    mask = np.where(fg_mask, fg_mask, 2)
    mask = np.where(bg_mask, 0, mask).copy()
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    mask, _, _ = cv.grabCut(img[::-1], mask, None, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_MASK)
    fg_mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    return hv.RGB(fg_mask[:, :, np.newaxis][::-1] * img[:, :, ::-1], bounds=bounds)

In [ ]:
%%opts RGB [width=400 height=500 xaxis=None yaxis=None] Polygons (line_width=5 fill_alpha=0 line_color=hv.Cycle())
%%opts Layout [merge_tools=False]
bg_paths = hv.Polygons([]) 
fg_paths = hv.Polygons([])
draw_fg = PolyDraw(source=fg_paths, rename={'data': 'fg_data'})
draw_bg = PolyDraw(source=bg_paths, rename={'data': 'bg_data'})
edit = PolyEdit(source=bg_paths)
edit2 = PolyEdit(source=fg_paths)

dmap = hv.DynamicMap(extract_foreground, streams=[draw_fg, draw_bg])

regrid(rgb) * fg_paths * bg_paths + regrid(dmap)

## Regridding

In [ ]:
file_path = '/Users/philippjfr/datashader/examples/data/MERCATOR_LC80210392016114LGN00_B%s.TIF'
bands = list(range(1, 12)) + ['QA']
bands = [xr.open_rasterio(file_path%band).load()[0] for band in bands]

from datashader.utils import ngjit
nodata= 1

@ngjit
def normalize_data(agg):
    out = np.zeros_like(agg)
    min_val = 0
    max_val = 2**16 - 1
    range_val = max_val - min_val
    col, rows = agg.shape
    c = 40
    th = .125
    for x in range(col):
        for y in range(rows):
            val = agg[x, y]
            norm = (val - min_val) / range_val
            norm = 1 / (1 + np.exp(c * (th - norm))) # bonus
            out[x, y] = norm * 255.0
    return out

def combine_bands(r, g, b):
    xs, ys = r['x'], r['y']
    r, g, b = [ds.utils.orient_array(img) for img in (r, g, b)]
    a = (np.where(np.logical_or(np.isnan(r),r<=nodata),0,255)).astype(np.uint8)    
    r = (normalize_data(r)).astype(np.uint8)
    g = (normalize_data(g)).astype(np.uint8)
    b = (normalize_data(b)).astype(np.uint8)
    col, rows = r.shape
    return hv.RGB((xs, ys[::-1], r, g, b, a), vdims=list('RGBA'))

In [ ]:
%%opts RGB Curve [width=350 height=350 xaxis=None yaxis=None] {+framewise}

combos = pd.DataFrame([
    (4,3,2,"True color",""),
    (7,6,4,"Urban","False color"),
    (5,4,3,"Vegetation","Color Infrared"),
    (6,5,2,"Agriculture",""),
    (7,6,5,"Penetration","Atmospheric Penetration"),
    (5,6,2,"Healthy Vegetation",""),
    (5,6,4,"Land vs. Water",""),
    (7,5,3,"Atmosphere Removal","Natural With Atmospheric Removal"),
    (7,5,4,"Shortwave Infrared",""),
    (6,5,4,"Vegetation Analysis","")],
    columns=['R', 'G', 'B', 'Name', 'Description']).set_index(["Name"])
combos

def combo(name):
    c=combos.loc[name]
    return combine_bands(bands[c.R-1],bands[c.G-1],bands[c.B-1]).relabel(name)

(combo("Urban") + combo("Vegetation") + combo("Agriculture") + combo("Land vs. Water")).cols(2)